In [1]:
import os
import sys
import re
sys.path.append("../")

from utils.io import read_csv, load_json, write_to

In [2]:
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)
    return rstring

def get_sighan_from_json():

    all_data = {
        "train":None,
        "dev":None,
        "test":None,
        "test14":None,
        "test15":None,
    }
    data_dir = "../data/rawdata/sighan/csc/"

    train_file1 = os.path.join(data_dir, "train_dev.json")
    train_file2 = os.path.join(data_dir, "train131415.json") 
    test14_file = os.path.join(data_dir, "test14.json")
    test15_file = os.path.join(data_dir, "test15.json")

    all_data["train"] = load_json(train_file1)
    all_data["train"].extend(load_json(train_file2))

    all_data["train"] = all_data["train"]

    all_data["valid14"] = load_json(test14_file)
    all_data["valid"] = load_json(test15_file)
    #all_data["test"].extend(load_json(test15_file))

    return all_data

def json2list(data, need_preprocess):
    source, target, ids = [], [], []

    for element in data:

        if need_preprocess:
            source.append(element["original_text"])
            target.append(element["correct_text"])
            ids.apoend(element["wrong_ids"])
        else:
            source.append(strQ2B((element["original_text"])))
            target.append(strQ2B((element["correct_text"])))
            ids.append(element["wrong_ids"])

    return source, target, ids

In [4]:
data = get_sighan_from_json()

source, target, ids = json2list(data["valid"], False)

raw_bert_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_base_std.epoch10.bs128/generated_predictions.txt")

#proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bshert_Flat_std_char.epoch10.bs16/generated_predictions.txt")
proto_preds = read_csv("/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan/bert_MaskedLM_v2_std.epoch10.bs32/generated_predictions.txt")

for i in range(len(source)):
    raw_bert_preds[i] = "".join(raw_bert_preds[i].split())[:len(source[i])]
    proto_preds[i] = "".join(proto_preds[i].split())[:len(source[i])] 

In [5]:
print(len(source), len(raw_bert_preds), len(target), len(proto_preds))

1100 1100 1100 1100


In [11]:
write_to("./newest_preds.txt", "\n".join(proto_preds))

In [6]:
need_cor = []
no_need_cor = []

def get_cor(source, target):
    return [ (j, target[j]) for j in range(len(target)) if target[j] != source[j]]


truth_cor_host, bert_cor_host, proto_cor_host = [], [], [] 

for i in range(len(source)):
    truth_cor, bert_cor, proto_cor = get_cor(source[i], target[i]), get_cor(source[i], raw_bert_preds[i]), get_cor(source[i], proto_preds[i])
    
    truth_cor_host.append(truth_cor)
    bert_cor_host.append(bert_cor)
    proto_cor_host.append(proto_cor)
    

In [7]:
print(truth_cor_host[0])

[]


In [8]:
joint = ""

for i in range(len(source)):
    first = source[i] + "\n"
    second = ["  "] * len(source[i]) + ["\n"]
    if truth_cor_host[i]:
        for cor in truth_cor_host[i]:
            second[cor[0]] = cor[1]
    
    third = ["  "] * len(source[i]) + ["\n"]
    if bert_cor_host[i]:
        for cor in bert_cor_host[i]:
            third[cor[0]] = cor[1]

    fourth = ["  "] * len(source[i]) + ["\n"]
    if proto_cor_host[i]:
        for cor in proto_cor_host[i]:
            fourth[cor[0]] = cor[1]
    joint += "".join(first) + "".join(second) + "".join(third) + "".join(fourth) + "\n"


write_to("tmp_analysis_bert_MLM_v2.txt", joint)
